In [1]:
# Codigo para gerar os dados mocados

import json
import random
import datetime
import os
import math

output_path = "./data/turtle_sensors.jsonl"
os.makedirs("./data", exist_ok=True)

NUM_ROWS = 10_000

def generate_temperature(t):
    minute_of_day = t % 1440  
    daily_cycle = 0.5 * math.sin(2 * math.pi * (minute_of_day / 1440))

    long_term_rise = 4 * (t / NUM_ROWS)

    base_temp = random.uniform(17, 22)  # base for nights
    temp = base_temp + daily_cycle * 10 + long_term_rise

    return round(min(max(temp, 17), 34), 2)  # clamp between 17 and 34


def generate_movement(t):
    if t < NUM_ROWS * 0.75:
        movement_intensity = 0.02  # very calm
    elif t < NUM_ROWS * 0.9:
        movement_intensity = 0.08  # increasing
    else:
        movement_intensity = 0.4  # babies starting to break shells

    ax = random.uniform(-movement_intensity, movement_intensity)
    ay = random.uniform(-movement_intensity, movement_intensity)
    az = 1 + random.uniform(-movement_intensity, movement_intensity)  # gravity reference

    gx = random.uniform(-movement_intensity, movement_intensity)
    gy = random.uniform(-movement_intensity, movement_intensity)
    gz = random.uniform(-movement_intensity, movement_intensity)

    return (
        round(ax, 4), round(ay, 4), round(az, 4),
        round(gx, 4), round(gy, 4), round(gz, 4)
    )


with open(output_path, "w") as f:
    start_time = datetime.datetime.now()

    for t in range(NUM_ROWS):
        temp = generate_temperature(t)
        ax, ay, az, gx, gy, gz = generate_movement(t)

        reading = {
            "timestamp": (start_time + datetime.timedelta(seconds=t)).isoformat(),
            "temperature_c": temp,
            "accelerometer": {"ax": ax, "ay": ay, "az": az},
            "gyroscope": {"gx": gx, "gy": gy, "gz": gz}
        }

        f.write(json.dumps(reading) + "\n")

print(f"Generated {NUM_ROWS} sensor readings in:", output_path)

Generated 10000 sensor readings in: ./data/turtle_sensors.jsonl


In [ ]:
%pip install -U dspy

In [2]:
# Carregamento do corpus

import json
from sentence_transformers import SentenceTransformer

with open("./data/turtle_sensors.jsonl") as f:
    corpus = []
    for line in f:
        obj = json.loads(line)
        
        text = (
            f"timestamp: {obj['timestamp']}. "
            f"temperature: {obj['temperature_c']}C. "
            f"accelerometer ax={obj['accelerometer']['ax']}, "
            f"ay={obj['accelerometer']['ay']}, "
            f"az={obj['accelerometer']['az']}. "
            f"gyroscope gx={obj['gyroscope']['gx']}, "
            f"gy={obj['gyroscope']['gy']}, "
            f"gz={obj['gyroscope']['gz']}."
        )
        corpus.append(text)

print("Loaded", len(corpus), "sensor readings.")

d:\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loaded 10000 sensor readings.


In [4]:

import dspy

# criação do embedder + retriever
model = SentenceTransformer("sentence-transformers/static-retrieval-mrl-en-v1", device="cpu")
embedder = dspy.Embedder(model.encode)
search = dspy.retrievers.Embeddings(embedder=embedder, corpus=corpus, k=10)


In [5]:
# Configurando a LM

# %pip install python-dotenv
from dotenv import load_dotenv
import os

gemini_api_key = os.getenv("GEMINI_API_KEY")

print("Loaded:", gemini_api_key[:5] + "..." if gemini_api_key else "Not found")

lm = dspy.LM("gemini/gemini-2.5-flash", api_key=gemini_api_key)
dspy.configure(lm=lm)

Loaded: AIzaS...


In [6]:
class SensorRAG(dspy.Module):
    def __init__(self):
        self.answer = dspy.ChainOfThought(
            "context, question -> response"
        )

    def forward(self, question):
        retrieved = search(question).passages
        return self.answer(context=retrieved, question=question)

sensor_rag = SensorRAG()

In [7]:
# 1a Pergunta
question = "Qual foi a temperatura média?"
rag_output = sensor_rag(question=question)
llm_answer = rag_output.response

# recuperar documentos do RAG corretamente
retrieved_docs = search(question).passages   # <-- AGORA É LISTA DE STRINGS

print("\nDocumentos recuperados:", len(retrieved_docs))


Documentos recuperados: 10


In [ ]:
# 2a Pergunta
sensor_rag(question="Quando ocorreram picos de atividade do acelerômetro?")

In [ ]:
# 3a Pergunta
sensor_rag(question="Qual foi a temperatura máxima registrada?")

In [ ]:
pip install deepeval

In [9]:
from deepeval.metrics import (
    ContextualPrecisionMetric,
    ContextualRecallMetric,
    ContextualRelevancyMetric
)
from deepeval.test_case import LLMTestCase
from deepeval import evaluate

In [12]:
from deepeval.metrics import (
    ContextualPrecisionMetric,
    ContextualRecallMetric,
    ContextualRelevancyMetric
)
from deepeval.test_case import LLMTestCase
from deepeval import evaluate
from deepeval.models import DeepEvalOllamaModel
import re

# ---- PERGUNTA ----
question = "Qual foi a temperatura média?"
rag_output = sensor_rag(question=question)
llm_answer = rag_output.response

# ---- RECUPERAR DOCUMENTOS ----
retrieved_docs = search(question).passages
print("\nRetrieved docs:", len(retrieved_docs))

# ---- CALCULAR A RESPOSTA ESPERADA (GROUND TRUTH) ----
temps = []
for text in corpus:
    m = re.search(r"temperature: ([0-9.]+)C", text)
    if m:
        temps.append(float(m.group(1)))

expected_output = f"A temperatura média correta é {sum(temps)/len(temps):.3f}C"

# ---- TEST CASE ----
test_case = LLMTestCase(
    input=question,
    actual_output=llm_answer,
    expected_output=expected_output,
    context=retrieved_docs,
    retrieval_context=retrieved_docs  # ← IMPORTANTE para evitar MissingTestCaseParamsError
)

# ---- MODELO OPEN-SOURCE PARA AVALIAÇÃO ----
eval_model = DeepEvalOllamaModel(model="llama3.1")

# ---- MÉTRICAS ----
contextual_precision = ContextualPrecisionMetric(model=eval_model)
contextual_recall = ContextualRecallMetric(model=eval_model)
contextual_relevancy = ContextualRelevancyMetric(model=eval_model)

# ---- EXECUTAR AVALIAÇÃO ----
results = evaluate(
    test_cases=[test_case],
    metrics=[contextual_precision, contextual_recall, contextual_relevancy]
)

# ---- RESULTADOS ----
print("\n--- RESULTADOS ---")
for r in results:
    print(r)


ImportError: cannot import name 'DeepEvalOllamaModel' from 'deepeval.models' (d:\Python310\lib\site-packages\deepeval\models\__init__.py)